# Categorical 
1. fill empty cells 
2. onehot Pclass ||
3. extract and cut the name ||
4. onehot Name ||
5. onehot Sex ||
6. onehot embarked ||
7. Extract the SibSp and Parch to Alone ||
8. onehot Alone ||
9. function to Age if add on cat or no
10. function to add fare or no

# Numerical
1. Fill empty cells
2. function to add age or no
3. function to add Fare or no

In [80]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer

In [81]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
combine = [train_df, test_df]

In [82]:
titanic_labels = train_df['Survived']
train_df.drop('Survived', axis=1, inplace=True)

In [83]:
train_df.describe(include=['O'])

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Zimmerman, Mr. Leo",male,347082,B96 B98,S
freq,1,577,7,4,644


In [84]:
for data in combine:
    data.drop(labels=["PassengerId", "Ticket"], axis=1, inplace=True)

In [85]:
def isAlone(X):
    X["Alone"] = X["SibSp"] + X["Parch"]
    X.Alone = X.Alone.apply(lambda x: 1 if x == 0 else 0)
    X.drop(['SibSp', 'Parch'], axis=1, inplace=True)

In [86]:
isAlone(train_df)

In [87]:
def extractName(X):
    X['Title'] = X.Name.str.extract('([A-Za-z]+)\.', expand = False)
    X.drop('Name', axis = 1, inplace=True)

In [88]:
extractName(train_df)

In [89]:
unique = list(train_df.Title.unique())

In [90]:
train_df.Title.value_counts()

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Major         2
Mlle          2
Col           2
Don           1
Jonkheer      1
Mme           1
Ms            1
Capt          1
Lady          1
Countess      1
Sir           1
Name: Title, dtype: int64

In [91]:
train_df.Title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'Countess',
       'Jonkheer'], dtype=object)

In [92]:
res = train_df.Title.value_counts() <= 2
rare = np.array(train_df.Title.value_counts().index.tolist())[res]


In [93]:
train_df.Title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'Countess',
       'Jonkheer'], dtype=object)

In [45]:
rare

array(['Major', 'Mlle', 'Col', 'Don', 'Jonkheer', 'Mme', 'Ms', 'Capt',
       'Lady', 'Countess', 'Sir'], dtype='<U8')

In [94]:
train_df.Title = train_df.Title.replace(rare, 'Rare')
train_df.Title = train_df.Title.replace('Mlle', 'Miss')
train_df.Title = train_df.Title.replace('Ms', 'Miss')
train_df.Title = train_df.Title.replace('Mme', 'Mrs')

In [95]:
train_df.Title.value_counts()

Mr        517
Miss      182
Mrs       125
Master     40
Rare       14
Dr          7
Rev         6
Name: Title, dtype: int64

In [96]:
train_df

,Pclass,Sex,Age,Fare,Cabin,Embarked,Alone,Title
0,3,male,22.0,7.2500,NaN,S,0,Mr
1,1,female,38.0,71.2833,C85,C,0,Mrs
2,3,female,26.0,7.9250,NaN,S,1,Miss
3,1,female,35.0,53.1000,C123,S,0,Mrs
4,3,male,35.0,8.0500,NaN,S,1,Mr
...,...,...,...,...,...,...,...,...
886,2,male,27.0,13.0000,NaN,S,1,Rev
887,1,female,19.0,30.0000,B42,S,1,Miss
888,3,female,NaN,23.4500,NaN,S,0,Miss
889,1,male,26.0,30.0000,C148,C,1,Mr


In [98]:
df_cat = train_df[['Pclass', 'Sex', 'Embarked', 'Alone', 'Title']]
df_num = train_df[['Age', 'Fare']]

In [99]:
encoded = OneHotEncoder(handle_unknown='ignore')

In [100]:
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('onehot', encoded),
])

df_cat_tr = cat_pipeline.fit_transform(df_cat)

In [101]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler()),
])

df_num_tr = num_pipeline.fit_transform(df_num)

In [103]:
num_attribs = list(df_num)
cat_attribs = list(df_cat)

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', cat_pipeline, cat_attribs),
])

df_prepared = full_pipeline.fit_transform(train_df)

In [105]:
df_prepared.shape

(891, 19)

In [106]:
log_reg = LogisticRegression()
log_reg.fit(df_prepared, titanic_labels)

LogisticRegression()

In [107]:
from sklearn.metrics import mean_squared_error
import numpy as np

titanic_predictions = log_reg.predict(df_prepared)


In [108]:
results = titanic_predictions == titanic_labels

In [109]:
results.value_counts()

True     725
False    166
Name: Survived, dtype: int64